In [1]:
from __future__ import print_function, division

# import sys
# sys.path.append(root_folder)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib

from models.convengers import Thor, IronMan
from models.solver import NickFury

from utils import dictionary, visualizations

import GPUtil

plt.ion()   # interactive mode

%load_ext autoreload
%autoreload 2

In [2]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 96% |
|  1 |  0% | 91% |
|  2 |  0% | 99% |
|  3 | 98% | 95% |


In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [4]:
dataset_folder = "data/tiny-imagenet-200/"
data_dir = pathlib.Path(dataset_folder)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
image_datasets = {x: datasets.ImageFolder(data_dir / x, data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=20, shuffle=True, num_workers=4, pin_memory=False) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [6]:
with torch.cuda.device(torch.cuda.current_device()):
    torch.cuda.empty_cache()
    
resnet_new = Thor(requires_grad=True)
resnet_new = resnet_new.to(device)
ResNet_solver = NickFury(resnet_new, dataloaders, dataset_sizes)

resnet_criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_new.parameters(), lr=0.001)
resnet_exp_lr_scheduler = lr_scheduler.StepLR(resnet_optimizer, step_size=7, gamma=0.1)

In [8]:
resnet_loss_history = ResNet_solver.train(resnet_optimizer, resnet_criterion, resnet_exp_lr_scheduler, device)

Epoch 0:


NameError: name 'label' is not defined

In [12]:
with torch.cuda.device(torch.cuda.current_device()):
    torch.cuda.empty_cache()
    
inception_new = IronMan()
inception_new = inception_new.to(device)

inception_solver = NickFury(inception_new, dataloaders, dataset_sizes)

inception_criterion = nn.CrossEntropyLoss()
inception_optimizer = optim.Adam(inception_new.parameters(), lr=0.001)
inception_exp_lr_scheduler = lr_scheduler.StepLR(inception_optimizer, step_size=7, gamma=0.1)

In [ ]:
inception_loss_history = ResNet_solver.train(inception_optimizer, inception_criterion, inception_exp_lr_scheduler)

In [22]:
images, labels = next(iter(dataloaders["val"]))
resnet_new.to("cpu")
inception_new.to("cpu")
with torch.no_grad():
    resnet_preds = resnet_new(images)
    inception_preds = inception_new(images)

print(labels)
print(torch.argmax(resnet_preds,dim=1))
print(torch.argmax(inception_preds,dim=1))

tensor([145, 120,  44,  32,  68, 139, 148,  99,  29,  74, 102, 133, 150,  79,
        156,  53, 131,  75,  69,  44])
tensor([145, 120,  44,  16, 171, 139,  41,  99,  35, 158, 102, 121, 152,  79,
        170,  53, 131,  84,  69,  44])
tensor([ 64, 120,  44,  16, 171,  99,  11, 109,  35, 158, 102, 133,  16, 173,
        143,  53, 177,  75,  69,  44])
